## NBA Salary Analysis: How Salary is affected by Player Performance


In [1]:
# connecting google drive to colab notebook, ensure path (defined in next code block) is in your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# imports
import pandas as pd
import altair as alt

path = '/content/drive/MyDrive/593 Project Folder/data/'
alt.data_transformers.disable_max_rows() # our merged dataset is slightly above the default allowed rows in altair

DataTransformerRegistry.enable('default')

In [3]:
# cleaning the salary dataset, done by Alex DeLoach
nba_salaries = pd.read_csv(path + 'salaries_1985to2018.csv')
nba_players = pd.read_csv(path + 'players.csv')
nba_salaries.head()

,index,league,player_id,salary,season,season_end,season_start,team
0,0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [4]:
# preview players data
nba_players.head()

,index,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,1,abdulza01,"April 7, 1946","Brooklyn, New York",1.2,42.8,NaN,72.8,505,15.1,...,5th overall,1st round,Cincinnati Royals,1968,6-9,"John Jay in Brooklyn, New York",Zaid Abdul-Aziz,Power Forward and Center,Right,235lb
2,2,abdulka01,"April 16, 1947","New York, New York",3.6,55.9,5.6,72.1,1560,24.6,...,1st overall,1st round,Milwaukee Bucks,1969,7-2,"Power Memorial in New York, New York",Kareem Abdul-Jabbar,Center,Right,225lb
3,3,abdulma02,"March 9, 1969","Gulfport, Mississippi",3.5,44.2,35.4,90.5,586,15.4,...,3rd overall,1st round,Denver Nuggets,1990,6-1,"Gulfport in Gulfport, Mississippi",Mahmoud Abdul-Rauf,Point Guard,Right,162lb
4,4,abdulta01,"November 3, 1974","Maisons Alfort, France",1.1,41.7,23.7,70.3,236,11.4,...,11th overall,1st round,Sacramento Kings,1997,6-6,"Lycee Aristide Briand in Evreux, France",Tariq Abdul-Wahad,Shooting Guard,Right,223lb


For this analysis, we will be mainly focusing on the data found in the 'nba_salaries' dataset. We will be using only the '_id' and 'name' columns from the 'nba_players' dataset for formatting/merging purposes.

In [5]:
# check for missing name or player ID values
nba_players[['_id', 'name']].isnull().sum()

_id     0
name    0
dtype: int64

In [6]:
# rename ID column to make merge possible
nba_players.rename(columns = {'_id': 'player_id'},
                   inplace = True)

# merge nba_player name to the nba_salaries dataset
salary = nba_salaries.merge(nba_players[['player_id', 'name']],
                            on = 'player_id')
salary.head()

,index,league,player_id,salary,season,season_end,season_start,team,name
0,0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,Alaa Abdelnaby
1,1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,Alaa Abdelnaby
2,2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics,Alaa Abdelnaby
3,3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics,Alaa Abdelnaby
4,4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings,Alaa Abdelnaby


Channing Frye was traded mid-season (2015-16) from the Orlando Magic to Cleveland Cavaliers, which shows that the dataset has each player on the team that they last played for in a season. This is important for merging with the per season stats. We show this in the code below.

In [7]:
salary[(salary['name'] == 'Channing Frye') & (salary['season_end'] == 2016)]

,index,league,player_id,salary,season,season_end,season_start,team,name
4324,4324,NBA,fryech01,7807579,2015-16,2016,2015,Cleveland Cavaliers,Channing Frye


In [8]:
# select necessary columns & rename season column
# per season stats are based on year the season ended in
columns = ('salary', 'season_end', 'team', 'name')
salary_df = salary.loc[:, columns]

salary_df.rename(columns = {'season_end': 'Year',
                            'team': 'Tm',
                            'name': 'Name',
                            'salary': 'Salary'}, inplace = True)

salary_df.head()

,Salary,Year,Tm,Name
0,395000,1991,Portland Trail Blazers,Alaa Abdelnaby
1,494000,1992,Portland Trail Blazers,Alaa Abdelnaby
2,500000,1993,Boston Celtics,Alaa Abdelnaby
3,805000,1994,Boston Celtics,Alaa Abdelnaby
4,650000,1995,Sacramento Kings,Alaa Abdelnaby


In [9]:
# filter for years 1998-2018
salary_df = salary_df[salary_df['Year'] >= 1998]

salary_df.head()

,Salary,Year,Tm,Name
16,3300000,1998,Sacramento Kings,Mahmoud Abdul-Rauf
17,798500,2001,Vancouver Grizzlies,Mahmoud Abdul-Rauf
18,1226880,1998,Sacramento Kings,Tariq Abdul-Wahad
19,1411000,1999,Sacramento Kings,Tariq Abdul-Wahad
20,1594920,2000,Denver Nuggets,Tariq Abdul-Wahad


In [10]:
# check for missing values
salary_df.isnull().sum()

Salary    0
Year      0
Tm        0
Name      0
dtype: int64

In [11]:
# team names will need to be adjusted to abbreviations
teams = list(salary_df['Tm'].unique())
team_abbreviations = {'Sacramento Kings': 'SAC',
                      'Vancouver Grizzlies': 'VAN',
                      'Denver Nuggets': 'DEN',
                      'Dallas Mavericks': 'DAL',
                      'Atlanta Hawks': 'ATL',
                      'Portland Trail Blazers': 'POR',
                      'Oklahoma City Thunder': 'OKC',
                      'Detroit Pistons': 'DET',
                      'Los Angeles Clippers': 'LAC',
                      'Toronto Raptors':'TOR',
                      'New York Knicks': 'NYK',
                      'Brooklyn Nets': 'BKN',
                      'New Jersey Nets': 'NJN',
                      'Memphis Grizzlies':'MEM',
                      'Miami Heat': 'MIA',
                      'Golden State Warriors': 'GSW',
                      'Houston Rockets': 'HOU',
                      'Charlotte Bobcats': 'CHB',
                      'Orlando Magic':'ORL',
                      'Minnesota Timberwolves':'MIN',
                      'San Antonio Spurs': 'SAS',
                      'New Orleans Pelicans':'NO',
                      'Philadelphia 76ers':'PHI',
                      'Washington Wizards':'WSH',
                      'New Orleans Hornets': 'NOH',
                      'Milwaukee Bucks':'MIL',
                      'Chicago Bulls':'CHI',
                      'Cleveland Cavaliers':'CLE',
                      'Indiana Pacers':'IND',
                      'Seattle SuperSonics': 'SEA',
                      'Boston Celtics':'BOS',
                      'Utah Jazz':'UTA',
                      'Phoenix Suns':'PHX',
                      'New Orleans/Oklahoma City Hornets': 'NOK',
                      'Charlotte Hornets':'CHA',
                      'Los Angeles Lakers':'LAL'
                     }


salary_df['Tm'] = salary_df['Tm'].replace(team_abbreviations)
salary_df.head()

<ipython-input-11-6bb6e53a1ae8>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df['Tm'] = salary_df['Tm'].replace(team_abbreviations)


,Salary,Year,Tm,Name
16,3300000,1998,SAC,Mahmoud Abdul-Rauf
17,798500,2001,VAN,Mahmoud Abdul-Rauf
18,1226880,1998,SAC,Tariq Abdul-Wahad
19,1411000,1999,SAC,Tariq Abdul-Wahad
20,1594920,2000,DEN,Tariq Abdul-Wahad


In [12]:
# check for duplicates
dupes = salary_df[salary_df.duplicated() == True]
dupes

,Salary,Year,Tm,Name


In [13]:
# cleaning the player stats dataset, done by Charles Ye
player_df = pd.read_csv(path + "Seasons_Stats.csv")

player_df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


We will be looking at the most recent two decades of the dataset (1998 - 2018). We will also be filtering the dataset to require a minimum of 10 games played per season and 8 minutes per game during the season in order to remove outlier cases where a player suffers from poor health and/or is not a regular part of a team's rotation.

In [14]:
modern_df = player_df[player_df["Year"] >= 1998]

modern_games_df = modern_df[modern_df["G"] >= 10]

modern_games_df = modern_games_df[modern_games_df["MP"] > 0]
modern_games_df["MPG"] = modern_games_df["MP"] / modern_games_df["G"]

modern_games_df = modern_games_df[modern_games_df["MPG"] >= 8]

modern_games_df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,MPG
13414,13414,1998.0,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,10.5,...,6.0,31.0,37.0,58.0,16.0,1.0,19.0,31.0,227.0,17.096774
13415,13415,1998.0,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,10.1,...,44.0,72.0,116.0,51.0,35.0,13.0,65.0,81.0,376.0,16.254237
13416,13416,1998.0,Shareef Abdur-Rahim,SF,21.0,VAN,82.0,82.0,2950.0,21.1,...,227.0,354.0,581.0,213.0,89.0,76.0,257.0,201.0,1829.0,35.975610
13417,13417,1998.0,Cory Alexander,PG,24.0,TOT,60.0,22.0,1298.0,15.2,...,17.0,129.0,146.0,209.0,70.0,11.0,112.0,98.0,488.0,21.633333
13418,13418,1998.0,Cory Alexander,PG,24.0,SAS,37.0,3.0,501.0,11.1,...,7.0,40.0,47.0,71.0,25.0,5.0,47.0,53.0,165.0,13.540541


We only want to keep the following data for our analysis: Year, Player name, Position, Team, Games played, Player Efficiency Rating, True shooting percentage, Offensive box plus minus, Defensive box plus minus, Box plus minus, Value over replacement, Field goal percentage, Free throw percentage, Total rebounds, Total assists, Total steals, Total blocks, Total points, Total 3 pointers.

In [15]:
columns = ["Year", "Player", "Pos", "Tm", "G", "PER", "TS%", "OBPM", "DBPM",\
            "BPM", "VORP", "FG%", "FT%", "TRB", "AST", "STL", "BLK", "PTS", "3P"]

filtered_modern_df = modern_games_df[columns]

filtered_modern_df.head()

,Year,Player,Pos,Tm,G,PER,TS%,OBPM,DBPM,BPM,VORP,FG%,FT%,TRB,AST,STL,BLK,PTS,3P
13414,1998.0,Mahmoud Abdul-Rauf,PG,SAC,31.0,10.5,0.405,-3.4,-3.7,-7.1,-0.7,0.377,1.000,37.0,58.0,16.0,1.0,227.0,5.0
13415,1998.0,Tariq Abdul-Wahad,SG,SAC,59.0,10.1,0.456,-4.2,-1.7,-5.9,-0.9,0.403,0.672,116.0,51.0,35.0,13.0,376.0,4.0
13416,1998.0,Shareef Abdur-Rahim,SF,VAN,82.0,21.1,0.562,2.6,-1.4,1.2,2.3,0.485,0.784,581.0,213.0,89.0,76.0,1829.0,21.0
13417,1998.0,Cory Alexander,PG,TOT,60.0,15.2,0.548,0.9,-0.8,0.1,0.7,0.428,0.784,146.0,209.0,70.0,11.0,488.0,66.0
13418,1998.0,Cory Alexander,PG,SAS,37.0,11.1,0.512,-1.7,0.2,-1.5,0.1,0.414,0.676,47.0,71.0,25.0,5.0,165.0,20.0


A few rows have incorrectly labeled free throw % where the player did not make a free throw but had attempted free throws. We will replace these null values with 0.

In [16]:
#replace instances of incorrectly labeled ft% null to 0
filtered_modern_df.loc[:,"FT%"] = filtered_modern_df["FT%"].fillna(0)
null_rows = filtered_modern_df[filtered_modern_df["FT%"] == 0]

#check for any remaining null values
null_values = filtered_modern_df.isnull().sum()

print(null_values)

Year      0
Player    0
Pos       0
Tm        0
G         0
PER       0
TS%       0
OBPM      0
DBPM      0
BPM       0
VORP      0
FG%       0
FT%       0
TRB       0
AST       0
STL       0
BLK       0
PTS       0
3P        0
dtype: int64


Some players have a secondary position where they spent time playing during their career. We'll be doing our analysis on each player's primary position.

In [17]:
#check for rows where a player has a secondary position
check_pos_df = filtered_modern_df[filtered_modern_df["Pos"].str.len() > 2]

def replace_pos(row):
  '''
  replace all instances of existing secondary positions so that only the primary position is shown
  '''
  if row["Pos"].startswith("C"):
      return "C"
  else:
      return row["Pos"][:2]

filtered_modern_df.loc[:,"Pos"] = filtered_modern_df.apply(replace_pos, axis = 1)

#check which unique values for positions now exist
unique_values = filtered_modern_df["Pos"].unique()
print(unique_values)

['PG' 'SG' 'SF' 'C' 'PF']


We will replace total season values for each player with the team name that they last played for, which is needed in joining with the salary dataset.

In [18]:
for (year, player), group in filtered_modern_df.groupby(["Year", "Player"]):
    #filter out rows where the player only played for one team during the season
    one_team = group[group["Tm"] != "TOT"]

    #find the last value of the groups(most recent team)
    if not one_team.empty:
        last_team = group[group["Tm"] != "TOT"].iloc[-1]["Tm"]

    #replace TOT with last value of group
        filtered_modern_df.loc[(filtered_modern_df["Year"] == year)\
                                & (filtered_modern_df["Player"] == player)\
                                & (filtered_modern_df["Tm"] == "TOT"), "Tm"] = last_team

#remove duplicate rows other than the total season row
filtered_modern_df = filtered_modern_df.drop_duplicates(subset = ["Year", "Player"], keep = "first")

#check for any remaining duplicate rows(Player + Year)
duplicate_rows = filtered_modern_df[filtered_modern_df.duplicated(subset = ["Player", "Year"], keep = False)]

print(duplicate_rows)

Empty DataFrame
Columns: [Year, Player, Pos, Tm, G, PER, TS%, OBPM, DBPM, BPM, VORP, FG%, FT%, TRB, AST, STL, BLK, PTS, 3P]
Index: []


In [19]:
# slight adjustments to cleaned data, then merging the datasets for analysis, done by Eric Zheng
filtered_modern_df = filtered_modern_df.rename(columns = {'Player': 'Name'})
filtered_modern_df['Year'] = filtered_modern_df['Year'].astype('int')
filtered_modern_df.head()

,Year,Name,Pos,Tm,G,PER,TS%,OBPM,DBPM,BPM,VORP,FG%,FT%,TRB,AST,STL,BLK,PTS,3P
13414,1998,Mahmoud Abdul-Rauf,PG,SAC,31.0,10.5,0.405,-3.4,-3.7,-7.1,-0.7,0.377,1.000,37.0,58.0,16.0,1.0,227.0,5.0
13415,1998,Tariq Abdul-Wahad,SG,SAC,59.0,10.1,0.456,-4.2,-1.7,-5.9,-0.9,0.403,0.672,116.0,51.0,35.0,13.0,376.0,4.0
13416,1998,Shareef Abdur-Rahim,SF,VAN,82.0,21.1,0.562,2.6,-1.4,1.2,2.3,0.485,0.784,581.0,213.0,89.0,76.0,1829.0,21.0
13417,1998,Cory Alexander,PG,DEN,60.0,15.2,0.548,0.9,-0.8,0.1,0.7,0.428,0.784,146.0,209.0,70.0,11.0,488.0,66.0
13420,1998,Ray Allen,SG,MIL,82.0,16.2,0.539,2.6,-0.8,1.8,3.2,0.428,0.875,405.0,356.0,111.0,12.0,1602.0,134.0


In [20]:
merged_df = pd.merge(filtered_modern_df,
                     salary_df,
                     on = ['Name', 'Tm', 'Year'])
merged_df.head()

,Year,Name,Pos,Tm,G,PER,TS%,OBPM,DBPM,BPM,VORP,FG%,FT%,TRB,AST,STL,BLK,PTS,3P,Salary
0,1998,Mahmoud Abdul-Rauf,PG,SAC,31.0,10.5,0.405,-3.4,-3.7,-7.1,-0.7,0.377,1.000,37.0,58.0,16.0,1.0,227.0,5.0,3300000
1,1998,Tariq Abdul-Wahad,SG,SAC,59.0,10.1,0.456,-4.2,-1.7,-5.9,-0.9,0.403,0.672,116.0,51.0,35.0,13.0,376.0,4.0,1226880
2,1998,Ray Allen,SG,MIL,82.0,16.2,0.539,2.6,-0.8,1.8,3.2,0.428,0.875,405.0,356.0,111.0,12.0,1602.0,134.0,2052360
3,1998,Derek Anderson,SF,CLE,66.0,15.3,0.531,0.4,0.7,1.0,1.4,0.408,0.873,187.0,227.0,86.0,13.0,770.0,17.0,1107240
4,1998,Nick Anderson,SF,ORL,58.0,19.6,0.529,3.4,-0.1,3.4,2.3,0.455,0.638,297.0,119.0,72.0,23.0,890.0,77.0,4000000


Now that our datasets are cleaned and merged, we can begin with some exploratory data analysis. For example, below would be a comparison of points between positions for each season, and we can do this for each stat.

In [21]:
pos_groups = merged_df.groupby(['Pos', 'Year'])
describe_pts = pos_groups['PTS'].describe().reset_index()
describe_pts.head()

,Pos,Year,count,mean,std,min,25%,50%,75%,max
0,C,1998,48.0,495.333333,362.298415,89.0,188.0,422.5,652.50,1574.0
1,C,1999,47.0,285.042553,218.399012,6.0,128.0,259.0,386.00,1061.0
2,C,2000,53.0,468.018868,341.175654,27.0,168.0,435.0,647.00,1463.0
3,C,2001,54.0,411.759259,294.132811,27.0,218.5,335.0,579.75,1250.0
4,C,2002,57.0,440.245614,347.717110,22.0,185.0,379.0,604.00,1779.0


In [22]:
describe_per = pos_groups['PER'].describe().reset_index()
describe_ts = pos_groups['TS%'].describe().reset_index()
describe_obpm = pos_groups['OBPM'].describe().reset_index()
describe_dbpm = pos_groups['DBPM'].describe().reset_index()
describe_bpm = pos_groups['BPM'].describe().reset_index()
describe_vorp = pos_groups['VORP'].describe().reset_index()
describe_fg = pos_groups['FG%'].describe().reset_index()
describe_ft = pos_groups['FT%'].describe().reset_index()
describe_trb = pos_groups['TRB'].describe().reset_index()
describe_ast = pos_groups['AST'].describe().reset_index()
describe_stl = pos_groups['STL'].describe().reset_index()
describe_blk = pos_groups['BLK'].describe().reset_index()
describe_3p= pos_groups['3P'].describe().reset_index()
describe_salary = pos_groups['Salary'].describe().reset_index()
# can use .head() like above to get a preview of any of these
describe_salary.head()

,Pos,Year,count,mean,std,min,25%,50%,75%,max
0,C,1998,48.0,2.306856e+06,2.221218e+06,242000.0,1101060.0,1800000.0,2933725.0,14285714.0
1,C,1999,47.0,2.901047e+06,2.604871e+06,287500.0,1025000.0,1923000.0,4233000.0,11000000.0
2,C,2000,53.0,3.421761e+06,2.865981e+06,301875.0,1289400.0,2530920.0,4988000.0,12250000.0
3,C,2001,54.0,3.711398e+06,2.675825e+06,316969.0,1694910.0,3085420.0,5475000.0,10130000.0
4,C,2002,57.0,3.344341e+06,2.673056e+06,288171.0,1067400.0,2970840.0,4538000.0,11250000.0


Next, we can generate some visualizations using Altair to show the results of our exploratory data analysis.

In [23]:
boxplot_data = merged_df[['Pos', 'Year', 'Salary']]

_98box_data = boxplot_data[boxplot_data['Year'] <= 2007]
_08box_data = boxplot_data[boxplot_data['Year'] >= 2008]

_98box = alt.Chart(_98box_data).mark_boxplot(color = 'blue').encode(
    x = alt.X('Pos:N', axis = alt.Axis(title = 'Position')),
    y = alt.Y('Salary:Q', scale = alt.Scale(domain = (0, 35000000)))
).properties(
      title='Salary Distribution by Position (1998 - 2007)',
      width = 150,
      height = 400
)

_08box = alt.Chart(_08box_data).mark_boxplot(color = 'blue').encode(
    x = alt.X('Pos:N', axis = alt.Axis(title = 'Position')),
    y = alt.Y('Salary:Q')
).properties(
      title='Salary Distribution by Position (2008 - 2017)',
      width = 150,
      height = 400
)

salary_boxplots = alt.hconcat(
    _98box,
    _08box,
)

salary_line_chart_by_pos= alt.Chart(describe_salary).mark_line().encode(
    x = alt.X('Year:O'),
    y = alt.Y('mean:Q', axis = alt.Axis(title = 'Mean')),
    color = alt.Color('Pos:N', scale = alt.Scale(scheme = 'category10'))
).properties(
    title = 'Mean Salary (by position), 1998 - 2017',
    height = 400
)
combined_chart = alt.hconcat(
    salary_boxplots,
    salary_line_chart_by_pos
)
combined_chart

alt.HConcatChart(...)

The box plots reveal a good number of outlier salaries. However, the median salary for each position are close together and fairly low. This is a point of interest to explore.

Let's see how each of the chosen recorded performance metrics correlate to a player's salary.

In [24]:
#remove non performance metrics from the correlation
exclude_columns = ["Year", "Name", "Pos", "Tm", "G"]

corr_cols = merged_df.select_dtypes(include = ["number"]).columns.difference(exclude_columns)

correlations = merged_df[corr_cols].corr()["Salary"].sort_values(ascending = False)

print(correlations)

Salary    1.000000
PTS       0.536186
VORP      0.492585
PER       0.489560
TRB       0.444590
BPM       0.442282
OBPM      0.400589
AST       0.363131
STL       0.330877
BLK       0.284918
3P        0.252881
TS%       0.234986
DBPM      0.178792
FG%       0.175843
FT%       0.137737
Name: Salary, dtype: float64


Do the correlation coefficients change based on a player's primary position? Lets group the dataframe by position and see how each position's salary correlation differs.

In [25]:
positional_correlations = merged_df.groupby("Pos")[corr_cols].corrwith(merged_df["Salary"])

positional_correlations

,3P,AST,BLK,BPM,DBPM,FG%,FT%,OBPM,PER,PTS,STL,Salary,TRB,TS%,VORP
Pos,,,,,,,,,,,,,,,
C,0.157500,0.482418,0.366512,0.375150,0.217121,0.132276,0.129963,0.345045,0.415141,0.510318,0.347378,1.0,0.476724,0.165485,0.448816
PF,0.268745,0.595322,0.326558,0.476482,0.239089,0.094614,0.229818,0.466732,0.505103,0.615542,0.455252,1.0,0.507201,0.188183,0.544105
PG,0.428690,0.540715,0.279028,0.510107,0.169218,0.230738,0.158861,0.509809,0.529897,0.555898,0.416215,1.0,0.517324,0.288961,0.551964
SF,0.374412,0.513011,0.198344,0.413488,0.081940,0.171941,0.202184,0.448171,0.476081,0.516623,0.338023,1.0,0.408538,0.262434,0.450734
SG,0.392623,0.513061,0.287253,0.438231,0.032757,0.197278,0.228662,0.485396,0.525936,0.567666,0.362765,1.0,0.398044,0.253813,0.487638


In order to better visualize the differences between these correlation coefficients, lets create a multi-faceted barchart for each performance statistic where we'll compare the correlation coefficients between the five primary positions.

In [26]:
def create_melted_df(df):
  '''
  transform df to long format to fit altair
  '''
  exclude_columns = ["Year", "Name", "Pos", "Tm", "G"]

  corr_cols = df.select_dtypes(include = ["number"]).columns.difference(exclude_columns)

  positional_correlations = df.groupby("Pos")[corr_cols].corrwith(df["Salary"])

  melted_df = positional_correlations.reset_index().melt(id_vars = ["Pos"], var_name = "Statistic", value_name = "Correlation")
  melted_df = melted_df[melted_df["Statistic"] != "Salary"]

  return melted_df

total_df = create_melted_df(merged_df)

#create the multi-faceted barcharts
barplot = alt.Chart(total_df).mark_bar().encode(
    x = alt.X("Pos:N", title = "Position"),
    y = alt.Y("Correlation:Q", title = "Correlation Coefficient"),
    color = alt.Color("Pos:N", title = "Position", scale = alt.Scale(scheme = "category10"))
).facet(
    facet= alt.Facet("Statistic:N",
                     title = "Statistics"),
    columns = 7,
    padding = 30,
    spacing = 30,
    title = alt.TitleParams(text = "Pearson Correlation Coefficient between Salary and Statistics",
                            anchor = "middle",
                            fontSize = 14
    )
).resolve_axis(
    x = 'independent',
    y = 'shared'
)

barplot

alt.FacetChart(...)

We can see that performance statistics such such as total Assists, PER(player efficiency rating), Points, and VORP(value over replacement) have the highest correlation coefficients with player salary.

While certain statistics have a stronger correlation than others, it appears that the correlation in general is positive for all the performance statistics that we're interested in. This makes sense since all 14 of the statistics that we are tracking are positively correlated with player performance(i.e. the higher the statistic, the better the player is performing).

Lets take a look at the highest level of earners for player salary within our dataset and see where their correlation coefficients per position
end up when compared to other top earners.

In [27]:
def find_upper_outliers(data, multiplier = 1.5): #Outlier implementation courtesy of Eric Zheng
  '''
  Gives upper outliers based on multiplier. Default multiplier results in all upper outliers.
  '''
  outliers = pd.DataFrame()
  for pos in data['Pos'].unique():
      pos_data = data[data['Pos'] == pos]
      Q1 = pos_data['Salary'].quantile(0.25)
      Q3 = pos_data['Salary'].quantile(0.75)
      IQR = Q3 - Q1
      upper_bound = Q3 + multiplier * IQR
      pos_outliers = pos_data[pos_data['Salary'] > upper_bound]
      outliers = pd.concat([outliers, pos_outliers])
  return outliers

outliers = find_upper_outliers(merged_df, 1.5) #filters the top 7% of per season earners

outlier_df = create_melted_df(outliers)

stat_cols = ["3P", "FT%", "TS%"]

outlier_df = outlier_df[outlier_df["Statistic"].isin(stat_cols)]

barplot = alt.Chart(outlier_df).mark_bar().encode(
    x = alt.X("Pos:N", title = "Position"),
    y = alt.Y("Correlation:Q", title = "Correlation Coefficient"),
    color = alt.Color("Pos:N", title = "Position", scale = alt.Scale(scheme = "category10"))
).facet(
    facet= alt.Facet("Statistic:N",
                     title = "Statistics",
                     header = alt.Header(labelFontSize = 14)),
    columns = 3,
    padding = 30,
    spacing = 30,
    title = alt.TitleParams(text = "Pearson Correlation Coefficient Between Salary and Statistics (Upper Outliers)",
                            anchor = "middle",
                            fontSize = 13
    )
).resolve_axis(
    x = 'independent',
    y = 'shared'
)

barplot

alt.FacetChart(...)

Looks like we have our first look into negative correlation coefficients. Interestingly enough, SG(Shooting Guards) actually have a negative correlation between total 3 pointers made and their salary. This can be attributed to the fact that pure 3 point shooting guards tend to have smaller contracts compared to the overabundance of superstar shooting guards such as Kobe Bryant and Dwayne Wade who were paid massive salaries and did not make as many 3s. Similarly, this phenomenon also explains why SGs have a negative correlation between their TS%(True Shooting Percentage) and their salaries. Superstar SGs like Kobe Bryant and Dwayne Wade took tough mid range shots which resulted in having a lower TS% compared to the average salaried 3 point focused shooting guard who tended to have higher TS%.

Another interesting correlation coefficient to look at is how Cs(Centers) had a negative correlation coefficient to their seasonal FT% while PFs(Power Forwards) had a positive correlation even though their playstyles were relatively similar throughout the bulk of the 2 decades. Some highly paid stars whos statistics definitely skewed the coefficient in this manner are Shaquille O'neal, Dwight Howard, and Rudy Gobert for centers with very low ft% and Kevin Garnett, Dirk Nowitzki, and Chris Bosh for power forwards with very high ft%.

Lets take a look at how the overall data's correlation coefficients changed from the first decade to the second.

In [28]:
_98data = merged_df[merged_df["Year"] <= 2007] #filter data to contain only the years 1998 to 2007

_98_df = create_melted_df(_98data)

barplot = alt.Chart(_98_df).mark_bar().encode(
    x = alt.X("Pos:N", title = "Position"),
    y = alt.Y("Correlation:Q", title = "Correlation Coefficient"),
    color = alt.Color("Pos:N", title = "Position", scale = alt.Scale(scheme = "category10"))
).facet(
    facet= alt.Facet("Statistic:N",
                     title = "Statistics",
                     header = alt.Header(labelFontSize = 14)),
    columns = 7,
    padding = 30,
    spacing = 30,
    title = alt.TitleParams(text = "Pearson Correlation Coefficient Between Salary and Statistics (1998 - 2007)",
                            anchor = "middle",
                            fontSize = 14
    )
).resolve_axis(
    x = 'independent',
    y = 'shared'
)

barplot

alt.FacetChart(...)

In [29]:
_08data = merged_df[merged_df["Year"] >= 2008] #filter data to contain only the years 2008 to 2017

_08_df = create_melted_df(_08data)

barplot = alt.Chart(_08_df).mark_bar().encode(
    x = alt.X("Pos:N", title = "Position"),
    y = alt.Y("Correlation:Q", title = "Correlation Coefficient"),
    color = alt.Color("Pos:N", title = "Position", scale = alt.Scale(scheme = "category10"))
).facet(
    facet= alt.Facet("Statistic:N",
                     title = "Statistics",
                     header = alt.Header(labelFontSize = 14)),
    columns = 7,
    padding = 30,
    spacing = 30,
    title = alt.TitleParams(text = "Pearson Correlation Coefficient Between Salary and Statistics (2008 - 2017)",
                            anchor = "middle",
                            fontSize = 14
    )
).resolve_axis(
    x = 'independent',
    y = 'shared'
)

barplot

alt.FacetChart(...)

We can definitely see some major differences between the correlation coefficients of the first decade compared to the second. Lets plot the difference between the two to better visualize how the NBA's valuation of certain positions and skillsets have changed over time.

In [30]:
_98_df = create_melted_df(_98data)
_08_df = create_melted_df(_08data)

diff_df = _08_df

#grabbing the difference between the correlation coefficients from 2008-2017 and 1998-2007
diff_df["Correlation"] = diff_df["Correlation"] - _98_df["Correlation"]

barplot = alt.Chart(diff_df).mark_bar().encode(
    y = alt.Y("Pos:N", title = "Position"),
    x = alt.X("Correlation:Q", title = "Difference in Correlation Coefficients"),
    color = alt.Color("Pos:N", title = "Position", scale = alt.Scale(scheme = "paired"))
).facet(facet= alt.Facet("Statistic:N",
                     title = "Statistics",
                     header = alt.Header(labelFontSize = 14)),
    columns = 2,
    padding = 30,
    spacing = 30,
    title = alt.TitleParams(text = "Difference In Pearson Correlation Coefficients Between Salary and Statistics (2008 - 2017 vs. 1998 - 2007)",
                            anchor = "middle",
                            fontSize = 14
    )
).resolve_axis(
    x = 'independent',
    y = 'shared'
)

barplot

alt.FacetChart(...)

While the changes in the correlation coefficients do not reflect any extremely strong changes, we can still observe some very obvious differences.

Small Forwards have seen a major increase in their correlation coefficients in nearly all statistics led by the supermax stars such as Lebron James, Carmelo Anthony, and Kevin Durant. Moreover, the archetype of "3 and D" - lengthy floor spacers who can knock down the 3 point shot as well as defend the opposing team's best player(usually one of the previously mentioned trio) - has gained massive valuations throughout the league as premier assets.

On the opposite side of the spectrum, we can see that Power Forwards have instead seen a relative decrease in most of their correlation coefficients(except in total 3 pointers) both as a result of the aging and retirement of the previous generation's superstars(Kevin Garnett, Dirk Nowitzki, Tim Duncan, etc.) and due to the fact that the role that the Power Forward once occupied was altered to become simply a big man who can shoot.

We can now look at how the metric that correlates the most with salary, points, differs between star players and role players. We will also see if this changes over time. To do this, we plot scatterplots comparing points and salary for each group and season.

In [31]:
data_98 = merged_df[merged_df['Year'] <= 2007]
data_08 = merged_df[merged_df['Year'] >= 2008]

outliers_98 = find_upper_outliers(data_98, 1.5)
outliers_08 = find_upper_outliers(data_08, 1.5)
all_outliers = pd.concat([outliers_98, outliers_08])
non_outliers = merged_df[~merged_df.index.isin(all_outliers.index)]

scatter_plot_outliers = alt.Chart(all_outliers).mark_circle(color = 'blue').encode(
    x='PTS:Q',
    y='Salary:Q',
    tooltip=['Name', 'PTS', 'Salary']
).properties(
    width=150,
    height=150
).facet(
    facet= alt.Facet('Year:O',
                     title = 'Outliers Only',
                     header = alt.Header(labelFontSize = 14,
                                         labelFontWeight = 'bold')),
    columns=4,
    padding = 30,
    title = alt.TitleParams(text = 'Points Scored (PTS) vs. Salary',
                            anchor = 'middle',
                            fontSize = 14
    )
).resolve_scale(
    x='independent',
    y='independent'
)

scatter_plot_outliers

alt.FacetChart(...)

In [32]:
scatter_plot_non_outliers = alt.Chart(non_outliers).mark_circle(color = 'blue').encode(
    x='PTS:Q',
    y='Salary:Q',
    tooltip=['Name', 'PTS', 'Salary']
).properties(
    width=150,
    height=150
).facet(
    facet= alt.Facet('Year:O',
                     title = 'Non-Outliers',
                     header = alt.Header(labelFontSize = 14,
                                         labelFontWeight = 'bold')),
    columns=4,
    padding = 30,
    title = alt.TitleParams(text = 'Points Scored (Pts) vs. Salary',
                            anchor = 'middle',
                            fontSize = 14
    )
).resolve_scale(
    x='independent',
    y='independent'
)

scatter_plot_non_outliers

alt.FacetChart(...)

For our next steps, addressing some of the study's limitations would be beneficial. Specifically, incorporating a reputation variable would require developing a scoring system to evaluate player reputations. Additionally, exploring other eras of the NBA, particularly earlier periods when the league was less established and salaries were much lower, could provide valuable insights. Lastly, while our study focuses on many commonly used statistics, examining newer metrics like the LEBRON stat could also yield interesting findings.
